In [ ]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../data/irtm.csv')

In [ ]:
df = df[np.array([len(str(c)) >= 100 for c in df['content']]) & np.array([cls in ['euclid', 'keter', 'safe'] for cls in df['object_class']])].reset_index(drop=True)
df.head()

In [ ]:
documents = df['content'].to_list()

In [ ]:
from pa2 import get_token
corpus = []
for i in range(len(documents)) :
    corpus.append(' '.join(get_token(documents[i])))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
TFIDF_VECTORIZER = TfidfVectorizer()
TF_IDF_vector = TFIDF_VECTORIZER.fit_transform(corpus)

In [ ]:
a = TF_IDF_vector.toarray()

In [ ]:
# sklearn.decomposition.MiniBatchSparsePCA
# from sklearn.datasets import make_friedman1
# from sklearn.decomposition import MiniBatchSparsePCA
from sklearn.manifold import TSNE
transformer = TSNE(n_components=2, n_iter=250, random_state=1126, n_jobs=-1)
# transformer = MiniBatchSparsePCA(n_components=2, batch_size=50,random_state=0)
transformer.fit(a)
MiniBatchSparsePCA(...)
X_transformed = transformer.transform(a)
X_transformed.shape

In [ ]:
plt.scatter(X_transformed[0],X_transformed[1])

In [ ]:
# DBSCAN
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(metric='euclidean' , eps = 1.2 , min_samples = 2)
dbscan.fit(TF_IDF_vector)

In [ ]:
dbscan.labels_

In [ ]:
from collections import Counter
print(Counter(dbscan.labels_))

In [ ]:
result = dbscan.labels_

In [ ]:
cluster = []
for i in range(0 , 254) :
    cluster.append([])
    for j in range(len(result)) :
        if(result[j] == (i - 1)) :
            cluster[i].append(j)

In [ ]:
for i in range(len(cluster)) :
    print(cluster[i])

In [ ]:
# K-means
from sklearn.cluster import KMeans
n_clusters = 20
cost = []
for i in range(2,n_clusters):    
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(TF_IDF_vector)
    cost.append(kmeans.inertia_)

In [ ]:
import matplotlib.pyplot as plt
x = range(2,n_clusters)
plt.plot(x,cost, 'bx-')

In [ ]:
final_n_clusters = 15
final_kmeans = KMeans(final_n_clusters)
final_kmeans.fit(TF_IDF_vector)
order_centroids = final_kmeans.cluster_centers_.argsort()[:,::-1]
Top_Term_Cluster = [[] for i in range(20)]
print("----  Top Terms of Each Cluster ----")
for i in range(final_n_clusters):
    # print("\n\nCluster %d keywords: " % (i+1))
    for ind in order_centroids[i, :20]:
        #print(TFIDF_vectorizer.get_feature_names()[ind])
        Top_Term_Cluster[i].append(TFIDF_VECTORIZER.get_feature_names()[ind])
for i in range(15):
    print(f'Top Terms for cluster {i+1}')
    print(Top_Term_Cluster[i])
    print('---------')